In [1]:
import numpy as np

In [34]:
train = np.loadtxt('Assignment1.2/train1.csv', delimiter=',', skiprows=1)
print(train.shape)
print(train[0])

(87595, 1184)
[1. 0. 0. ... 0. 0. 4.]


In [39]:
# append a column of ones to the train data and remove the last column as class_label
X = train[:, :-1]
one = np.ones((X.shape[0], 1))
X= np.hstack((one,X))
Y = train[:, -1]
print(X.shape, Y.shape)
unique, counts = np.unique(Y, return_counts=True)
print(unique, counts)

(87595, 1184) (87595,)
[1. 2. 3. 4.] [16131  1032 25406 45026]


In [40]:
W = np.zeros((X.shape[1], len(unique)), dtype=np.float64)
print(W.shape)

(1184, 4)


In [8]:
def g(W,x,j):
    z = np.dot(x, W)
    # z -= np.max(z)    # TODO: for numerical stability (might add this)
    sm = (np.exp(z[j]) / np.sum(np.exp(z)))
    return sm

In [12]:
def g(W,x,j):
    z = np.dot(x, W)
    # z -= np.max(z)    # TODO: for numerical stability (might add this)
    sm = (np.exp(z[j]) / np.sum(np.exp(z)))
    return sm

def loss(X,Y,W,counts):
    # X is n * m
    # Y is n * 1
    # W is m * k
    n = X.shape[0]
    k = W.shape[1]
    print(n,k)
    loss = 0
    for i in range(n):
        x = X[i]
        y = Y[i]
        for j in range (k):
            if (y == j):
                loss += (np.log(g(W, x, j)) / counts[j-1])
    loss /= -2*n
    return loss

print(loss(X,Y,W,counts))

87595 4
2.373927212376198e-05


In [41]:
def g(W, X):
    # print(W.shape, X.shape)
    z = np.dot(X, W)
    # print(z.shape)
    # print(z[:2])
    # z -= np.max(z, axis=1, keepdims=True)  # For numerical stability
    exp_scores = np.exp(z)
    # print(exp_scores.shape)
    # print(exp_scores[:2])
    softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    return softmax

def loss(X, Y, W, counts):
    n = X.shape[0]
    
    # Calculate softmax for all classes
    softmax = g(W, X)
    # print(softmax.shape)
    # print(softmax[:2])
    
    # Convert Y to integer type if necessary
    Y = Y.flatten().astype(int) -1  # 0-indexing
    
    # Extract the softmax probabilities for the correct class labels
    correct_class_probabilities = softmax[np.arange(n), Y]
    # print(correct_class_probabilities.shape)
    # print(correct_class_probabilities[:2])
    
    # Logarithm of the probabilities
    log_probs = np.log(correct_class_probabilities)
    
    # Calculate the weighted loss (using counts for balancing, if required)
    weights = 1 / counts[Y]
    # print(weights.shape)
    # print(weights[:2])
    loss = -np.sum(log_probs * weights) / n
    loss /= 2
    
    return loss

# print(loss(X, Y, W, counts))

In [74]:
def g(W, X):
    print("In g fxn")
    z = np.dot(X, W)
    z -= np.max(z, axis=1, keepdims=True)  # For numerical stability
    print(z.shape)
    print(z[:3])
    exp_scores = np.exp(z)
    print(exp_scores.shape)
    print(exp_scores[:3])
    softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    print(softmax.shape)
    print(softmax[:3])
    print("exiting g fxn")
    return softmax

def loss(X, Y, W, counts):
    print("In loss fxn")
    print(W[:3])
    n = X.shape[0]
    softmax = g(W, X)
    print(softmax.shape)
    print(softmax[:3])
    Y = Y.flatten().astype(int) - 1  # Convert to 0-indexed
    correct_class_probabilities = softmax[np.arange(n), Y]
    print(correct_class_probabilities.shape)
    print(correct_class_probabilities[:3])
    log_probs = np.log(correct_class_probabilities)
    weights = 1 / counts[Y]
    loss = -np.sum(log_probs * weights) / n
    loss /= 2  # If you want to add this factor
    print("exitint loss fxn")
    return loss

def compute_gradient(X, Y, W, counts):
    n, m = X.shape
    k = W.shape[1]
    softmax = g(W, X)
    # print("Softmax shape: ", softmax.shape)
    # print(softmax[:2])
    Y = Y.flatten().astype(int) - 1  # Convert to 0-indexed
    
    # Create a one-hot encoding of Y
    Y_one_hot = np.zeros((n, k))
    Y_one_hot[np.arange(n), Y] = 1
    # print("Y one hot shape: ", Y_one_hot.shape)
    # print(Y_one_hot[:2])
    
    # Compute the gradient
    weights = 1 / counts[Y]
    # print("Weights shape: ", weights.shape)
    # print(weights[:2])
    gradient = np.dot(X.T, (softmax - Y_one_hot) * weights[:, np.newaxis]) / (2*n)
    
    return gradient

def gradient_descent(X, Y, W, counts, learning_rate, iterations):
    for i in range(iterations):
        gradient = compute_gradient(X, Y, W, counts)
        # print(gradient.shape)
        # print(gradient[:4])
        W -= learning_rate * gradient  # Update rule
        print(W[:3])
        
        if i % 2 == 0:
            # Print the loss every 100 iterations
            current_loss = loss(X, Y, W, counts)
            print(f"Iteration {i}/{iterations}: Loss = {current_loss}")
    
    return W

X = train[:, :-1]
one = np.ones((X.shape[0], 1))
X= np.hstack((one,X))
Y = train[:, -1]
unique, counts = np.unique(Y, return_counts=True)
W = np.zeros((X.shape[1], len(unique)), dtype=np.float64)

W = gradient_descent(X, Y, W, counts, learning_rate=0.01, iterations=1)
# print(W[:4])

In g fxn
(87595, 4)
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
(87595, 4)
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
(87595, 4)
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]
exiting g fxn
[[ 1.09911699e-23 -5.50796240e-23  5.89166316e-24  4.49301203e-24]
 [-1.37509156e-09 -1.13442682e-10 -1.30177216e-09  2.79030641e-09]
 [-1.06401690e-09  2.15899451e-09 -2.21224561e-09  1.11726800e-09]]
In loss fxn
[[ 1.09911699e-23 -5.50796240e-23  5.89166316e-24  4.49301203e-24]
 [-1.37509156e-09 -1.13442682e-10 -1.30177216e-09  2.79030641e-09]
 [-1.06401690e-09  2.15899451e-09 -2.21224561e-09  1.11726800e-09]]
In g fxn
(87595, 4)
[[  66989.49334257   39615.46910484   91918.18354689 -198523.1459943 ]
 [ 109598.3013608    64820.83301065  150392.6856745  -324811.82004596]
 [  28432.61681618   16813.02775578   39011.80659712  -84257.45116909]]
(87595, 4)
[[inf inf inf  0.]
 [inf inf inf  0.]
 [inf inf inf  0.]]
(87595, 4)
[[nan nan nan  0.]
 [nan nan nan  0.]
 [nan nan nan 

/tmp/ipykernel_80498/3498699899.py:7: RuntimeWarning: overflow encountered in exp
  exp_scores = np.exp(z)
/tmp/ipykernel_80498/3498699899.py:10: RuntimeWarning: invalid value encountered in divide
  softmax = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
/tmp/ipykernel_80498/3498699899.py:27: RuntimeWarning: divide by zero encountered in log
  log_probs = np.log(correct_class_probabilities)


In [31]:
def gradient_descent(X, Y, W, counts, learning_rate=0.01, num_iters=100):
    n = X.shape[0]
    k = W.shape[1]
    
    for i in range(num_iters):
        # Calculate softmax for all classes
        softmax = g(W, X)
        
        # Convert Y to integer type if necessary
        Y = Y.flatten().astype(int) - 1  # 0-indexing
        
        # Extract the softmax probabilities for the correct class labels
        correct_class_probabilities = softmax[np.arange(n), Y]
        
        # Calculate the gradient
        softmax[np.arange(n), Y] -= 1
        dW = np.dot(X.T, softmax)
        
        # Update the weights
        W -= learning_rate * dW
        
        # Calculate the loss
        loss = -np.sum(np.log(correct_class_probabilities)) / n
        loss /= 2
        print('Iteration %d: Loss = %f' % (i, loss))
        
    return W

W = gradient_descent(X, Y, W, counts, learning_rate=0.01, num_iters=100)
# print(W)

Iteration 0: Loss = nan
Iteration 1: Loss = nan
Iteration 2: Loss = nan
Iteration 3: Loss = nan
Iteration 4: Loss = nan


IndexError: index -5 is out of bounds for axis 1 with size 4